# Script to evaluate survey responses using LLM



# Configuration
INITIALIZE spreadsheet_file
INITIALIZE questions_list
INITIALIZE llm_api_connection
INITIALIZE survey_data (rows 3 to n)



In [ ]:
import openpyxl
from openpyxl import load_workbook, Workbook

# INITIALIZE spreadsheet_file
# Option 1: Create a new workbook
# wb = Workbook()
# ws = wb.active
# spreadsheet_file = "questions_spreadsheet.xlsx"

# Option 2: Load existing workbook (uncomment to use)
spreadsheet_file = "questions_spreadsheet.xlsx"
try:
    wb = load_workbook(spreadsheet_file)
    ws = wb.active
    print(f"Loaded existing spreadsheet: {spreadsheet_file}")
except FileNotFoundError:
    wb = Workbook()
    ws = wb.active
    print(f"Created new spreadsheet: {spreadsheet_file}")

# INITIALIZE questions_list
questions_list = [
    "What is your name?",
    "What is your favorite color?",
    "What is your hobby?",
    # Add your questions here
]

print(f"Initialized {len(questions_list)} questions")

In [ ]:
# INITIALIZE llm_api_connection
# This example uses Anthropic's Claude API
# You can replace with OpenAI or other LLM providers

from anthropic import Anthropic
import os

# Initialize Anthropic client
# Make sure to set your API key: export ANTHROPIC_API_KEY='your-key-here'
try:
    anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
    print("Anthropic API client initialized")
except Exception as e:
    print(f"Warning: Could not initialize Anthropic client: {e}")
    anthropic_client = None

# Alternative: OpenAI
# from openai import OpenAI
# openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [ ]:
# Copy survey_data (rows 3 to n) from survey_spreadsheet to questions_spreadsheet
# This assumes you have a separate survey spreadsheet with responses

survey_spreadsheet_file = "survey_responses.xlsx"

def copy_survey_data(source_file, target_worksheet, start_row=3):
    """
    Copy survey data from source spreadsheet to target worksheet starting at start_row.
    
    Args:
        source_file: Path to the survey spreadsheet
        target_worksheet: The target openpyxl worksheet
        start_row: Starting row in target worksheet (default: 3)
    """
    try:
        survey_wb = load_workbook(source_file)
        survey_ws = survey_wb.active
        
        # Get the dimensions of source data
        max_row = survey_ws.max_row
        max_col = survey_ws.max_column
        
        # Copy data starting from row 3 in source to start_row in target
        row_offset = start_row - 3  # Adjust if source data starts at different row
        
        for row in range(3, max_row + 1):
            for col in range(1, max_col + 1):
                source_value = survey_ws.cell(row=row, column=col).value
                target_worksheet.cell(row=row + row_offset, column=col, value=source_value)
        
        print(f"Copied {max_row - 2} rows of survey data")
        survey_wb.close()
        
    except FileNotFoundError:
        print(f"Survey file '{source_file}' not found. Skipping data copy.")
    except Exception as e:
        print(f"Error copying survey data: {e}")

# Uncomment to copy survey data:
# copy_survey_data(survey_spreadsheet_file, ws)

# Step 1: Populate first row with questions




In [ ]:
def populate_questions(worksheet, questions_list):
    """
    Populate the first row with questions from questions_list.
    
    Args:
        worksheet: The openpyxl worksheet object
        questions_list: List of question strings to populate
    """
    # Start from column 1 (column A)
    for col_index, question in enumerate(questions_list, start=1):
        # Write each question to row 1
        worksheet.cell(row=1, column=col_index, value=question)
    
    print(f"Populated {len(questions_list)} questions in row 1")
    
# Test/Example usage:
# populate_questions(ws, questions_list)

# Step 2: Generate LLM answers for each question (row 2)




In [ ]:
# FUNCTION generate_llm_answers():
#     FOR each question IN row 1:
#         column_index = get_column_index(question)
        
#         # Prompt LLM to answer the question
#         prompt = "Answer the following question: " + question
#         llm_answer = call_llm_api(prompt)
        
#         # Write LLM answer to row 2
#         write_to_cell(row=2, col=column_index, value=llm_answer)
#     END FOR
# END FUNCTION

# Step 3: Create evaluation columns




In [ ]:
print("Question grader setup complete.")

: 

In [ ]:
# FUNCTION create_evaluation_columns():
#     question_count = count_columns_with_questions()
    
#     FOR i = 1 TO question_count:
#         # Insert new column after each question column
#         question_col = get_column_index_for_question(i)
#         evaluation_col = question_col + 1
        
#         insert_column(at=evaluation_col)
        
#         # Add header for evaluation column
#         evaluation_header = "Q" + i + "_evaluation"
#         write_to_cell(row=1, col=evaluation_col, value=evaluation_header)
#     END FOR
# END FUNCTION

# Step 4: Evaluate each human answer



In [ ]:
# FUNCTION evaluate_human_answers():
#     question_count = count_questions()
    
#     FOR question_num = 1 TO question_count:
#         # Get question and LLM answer
#         question_col = get_column_index_for_question(question_num)
#         evaluation_col = question_col + 1
        
#         question_text = read_cell(row=1, col=question_col)
#         known_answer = read_cell(row=2, col=question_col)
        
#         # Iterate through all human responses (rows 3 to n)
#         FOR row = 3 TO last_row_with_data:
#             human_answer = read_cell(row=row, col=question_col)
            
#             # Skip if human answer is empty
#             IF human_answer IS empty:
#                 CONTINUE
#             END IF
            
#             # Construct evaluation prompt
#             evaluation_prompt = """
#             Question: {question_text}
#             Known correct answer: {known_answer}
#             Human answer to evaluate: {human_answer}
            
#             Please evaluate the human answer compared to the known answer.
#             Provide a review including:
#             - Accuracy assessment
#             - Key points covered or missed
#             - Overall quality rating
#             """
            
#             # Get LLM evaluation
#             evaluation_result = call_llm_api(evaluation_prompt)
            
#             # Write evaluation to spreadsheet
#             write_to_cell(row=row, col=evaluation_col, value=evaluation_result)
            
#         END FOR
#     END FOR
# END FUNCTION


# Main execution


In [ ]:
# FUNCTION main():
#     # Step 1: Set up questions
#     populate_questions()
    
#     # Step 2: Get LLM answers
#     generate_llm_answers()
    
#     # Step 3: Create evaluation columns
#     create_evaluation_columns()
    
#     # Step 4: Evaluate all human answers
#     evaluate_human_answers()
    
#     # Save spreadsheet
#     save_spreadsheet()
    
#     PRINT "Evaluation complete!"
# END FUNCTION

# # Run the script
# main()